In [90]:
EMAIL_DOMAINS = [
    "gmail.com", "gmail.net", "gmail.org",
    "yahoo.net", "yahoo.com", "yahoo.org",
    "hotmail.net", "hotmail.org", "hotmail.com",
    "outlook.org", "outlook.com", "outlook.net"
]

EMAIL_ENDINGS = [".com", ".net", ".org"]

def validate_name(name, storage=None):
    if name.strip() == "":
        raise ValueError("Name cannot be empty")

    for ch in name:
        if ch in "!@#$%^&*(){}[]<>?/~`+=":
            raise ValueError("Name contains invalid characters")

    if storage is not None:
        for c in storage.get_all_contacts():
            if c.name.lower() == name.lower():
                raise ValueError("This name already exists")
    return True


def validate_phone(phone, storage=None):
    if not phone.isdigit():
        raise ValueError("Phone must contain only digits")

    if len(phone) != 10 and len(phone) != 11:
        raise ValueError("Phone must be 10 or 11 digits long")

    if storage is not None:
        if phone in storage.phone_index:
            raise ValueError("This phone number already exists")

    return True


def validate_email(email):
    if "@" not in email:
        raise ValueError("Email must contain '@'")

    valid_end = False
    for end in EMAIL_ENDINGS:
        if email.endswith(end):
            valid_end = True

    if not valid_end:
        raise ValueError("Email must end with .com, .net or .org")

    domain = email.split("@")[1]

    if domain not in EMAIL_DOMAINS:
        raise ValueError("Email domain '" + domain + "' not allowed")

    return True


def validate_address(address):
    if address.strip() == "":
        raise ValueError("Address cannot be empty")

    for ch in address:
        if ch in "*{}<>":
            raise ValueError("Address contains invalid characters")

    return True


KEY = 3

def encrypt(text):
    result = ""
    for ch in text:
        result += chr(ord(ch) + KEY)
    return result

def decrypt(text):
    result = ""
    for ch in text:
        result += chr(ord(ch) - KEY)
    return result


In [91]:
class Contact:
    def __init__(self, name, phone, email, address):
        validate_name(name)
        validate_phone(phone)
        validate_email(email)
        validate_address(address)

        self.name = name
        self.phone = phone
        self._email = encrypt(email)
        self._address = encrypt(address)

    def update(self, phone=None, email=None, address=None):
        if phone is not None:
            validate_phone(phone)
            self.phone = phone
        if email is not None:
            validate_email(email)
            self._email = encrypt(email)
        if address is not None:
            validate_address(address)
            self._address = encrypt(address)

    def get_display_email(self):
        return decrypt(self._email)

    def get_display_address(self):
        return decrypt(self._address)

    def __str__(self):
        return (
            f"Name: {self.name}\n"
            f"Phone: {self.phone}\n"
            f"Email: {self.get_display_email()}\n"
            f"Address: {self.get_display_address()}\n"
        )


In [92]:
class Storage:
    def __init__(self):
        self.contacts_list = []
        self.phone_index = {}

    def add(self, contact):
        if contact.phone in self.phone_index:
            raise Exception("Duplicate phone number")
        self.phone_index[contact.phone] = contact
        self.contacts_list.append(contact)

    def delete(self, phone):
        if phone not in self.phone_index:
            return False
        contact = self.phone_index.pop(phone)
        self.contacts_list.remove(contact)
        return True

    def update(self, phone, new_data):
        if phone not in self.phone_index:
            return False

        contact = self.phone_index[phone]

        if new_data.phone != phone:
            if new_data.phone in self.phone_index:
                raise Exception("Duplicate phone number (update blocked)")

        contact.update(
            phone=new_data.phone,
            email=new_data.get_display_email(),
            address=new_data.get_display_address()
        )

        if new_data.phone != phone:
            self.phone_index.pop(phone)
            self.phone_index[new_data.phone] = contact

        return True

    def search_by_phone(self, phone):
        if phone in self.phone_index:
            return self.phone_index[phone]
        return None

    def get_sorted_by_name(self):
        contacts = self.contacts_list[:]
        contacts.sort(key=lambda c: c.name)
        return contacts

    def get_sorted_by_phone(self):
        contacts = self.contacts_list[:]
        contacts.sort(key=lambda c: c.phone)
        return contacts

    def get_all_contacts(self):
        return self.contacts_list


In [93]:
class FileHandler:
    def __init__(self, storage):
        self.storage = storage

    def load_from_file(self, filename):
        loaded_count = 0
        try:
            with open(filename, "r") as file:
                for line in file:
                    line = line.strip()
                    if not line:
                        continue
                    parts = line.split(",")
                    if len(parts) != 4:
                        continue
                    name, phone, enc_email, enc_addr = parts
                    try:
                        email = decrypt(enc_email)
                        address = decrypt(enc_addr)
                        contact = Contact(name, phone, email, address)
                        try:
                            self.storage.add(contact)
                            loaded_count += 1
                        except:
                            continue
                    except:
                        continue
            print(f"{loaded_count} contacts loaded successfully.")
            return True
        except FileNotFoundError:
            print("File not found.")
            return False
        except Exception as e:
            print("Failed to load file:", e)
            return False


    def save_to_file(self, filename):
        try:
            file = open(filename, "w")
        except:
            return False

        for contact in self.storage.get_all_contacts():
            line = contact.name + "," + contact.phone + "," + contact._email + "," + contact._address + "\n"
            file.write(line)

        file.close()
        return True


In [94]:
def input_with_validation(prompt, validator, extra=None):
    while True:
        value = input(prompt)
        try:
            if extra is None:
                validator(value)
            else:
                validator(value, extra)
            return value
        except Exception as e:
            print("Error:", str(e))

In [95]:
class ContactManager:
    def __init__(self, storage, file_handler):
        self.storage = storage
        self.file_handler = file_handler

    def add_contact(self):
        try:
            name = input_with_validation("Name: ", validate_name, self.storage)
            phone = input_with_validation("Phone: ", validate_phone, self.storage)
            email = input_with_validation("Email: ", validate_email)
            address = input_with_validation("Address: ", validate_address)

            contact = Contact(name, phone, email, address)
            self.storage.add(contact)
            print("Contact added.")
        except Exception as e:
            print("Error:", str(e))

    def view_contacts(self):
        contacts = self.storage.get_sorted_by_name()
        if contacts == []:
            print("No contacts found.")
            return
        for contact in contacts:
            print(contact)

    def search_contact(self):
        print("\nSearch by:")
        print("1. Phone")
        print("2. Name")
        choice = input("Choice: ")

        if choice == "1":
            phone = input("Enter phone: ")
            contact = self.storage.search_by_phone(phone)
            if contact is not None:
                print(contact)
            else:
                print("Contact not found.")

        elif choice == "2":
            name = input("Enter name: ").lower()
            found = None
            for c in self.storage.get_all_contacts():
                if c.name.lower() == name:
                    found = c
                    break

            if found is not None:
                print(found)
            else:
                print("Contact not found.")

        else:
            print("Invalid choice.")

    def update_contact(self):
        old_phone = input("Phone to update: ")
        contact = self.storage.search_by_phone(old_phone)
        if contact is None:
            print("Contact not found.")
            return

        print("\nWhat do you want to update?")
        print("1. Phone")
        print("2. Email")
        print("3. Address")
        print("4. Update all")
        choice = input("Choice: ")

        new_phone = contact.phone
        new_email = contact.get_display_email()
        new_address = contact.get_display_address()

        try:
            if choice == "1":
                new_phone = input_with_validation("New phone: ", validate_phone, self.storage)

            elif choice == "2":
                new_email = input_with_validation("New email: ", validate_email)

            elif choice == "3":
                new_address = input_with_validation("New address: ", validate_address)

            elif choice == "4":
                new_phone = input_with_validation("New phone: ", validate_phone, self.storage)
                new_email = input_with_validation("New email: ", validate_email)
                new_address = input_with_validation("New address: ", validate_address)

            else:
                print("Invalid choice.")
                return

            updated_contact = Contact(contact.name, new_phone, new_email, new_address)
            self.storage.update(old_phone, updated_contact)
            print("Contact updated.")

        except Exception as e:
            print("Error:", str(e))

    def delete_contact(self):
        print("\nDelete by:")
        print("1. Phone")
        print("2. Name")
        choice = input("Choice: ")

        if choice == "1":
            phone = input("Enter phone: ")
            if self.storage.delete(phone):
                print("Contact deleted.")
            else:
                print("Contact not found.")

        elif choice == "2":
            name = input("Enter name: ").lower()
            found = None
            for c in self.storage.get_all_contacts():
                if c.name.lower() == name:
                    found = c
                    break

            if found is not None:
                self.storage.delete(found.phone)
                print("Contact deleted.")
            else:
                print("Contact not found.")

        else:
            print("Invalid choice.")

    def load_data(self):
        filename = input("Enter file name to load: ")
        result = self.file_handler.load_from_file(filename)
        if result:
            print("Data loaded successfully.")
        else:
            print("No file found or file corrupted.")

    def save_data(self):
        result = self.file_handler.save_to_file("contacts.txt")
        if result:
            print("Data saved successfully.")
        else:
            print("Failed to save data.")


In [96]:
class Menu:
    def __init__(self, manager):
        self.manager = manager

    def show_menu(self):
        print("\n===== CONTACT BOOK =====")
        print("1. Add Contact")
        print("2. View Contacts")
        print("3. Search Contact")
        print("4. Update Contact")
        print("5. Delete Contact")
        print("6. Load Data")
        print("7. Save Data")
        print("0. Exit")

    def handle_choice(self, choice):
        if choice == "1":
            self.manager.add_contact()
        elif choice == "2":
            self.manager.view_contacts()
        elif choice == "3":
            self.manager.search_contact()
        elif choice == "4":
            self.manager.update_contact()
        elif choice == "5":
            self.manager.delete_contact()
        elif choice == "6":
            self.manager.load_data()
        elif choice == "7":
            self.manager.save_data()
        elif choice == "0":
            print("Saving and exiting...")
            self.manager.save_data()
            return False
        else:
            print("Invalid choice!")
        return True

    def run(self):
        running = True
        while running:
            self.show_menu()
            user_choice = input("Choice: ")
            running = self.handle_choice(user_choice)


In [97]:
def create_sample_data(storage):
    print("\n--- Loading Sample Test Data ---")

    contacts = [
        ["Ayesha Khan", "03001234567", "ayesha@gmail.com", "House 12, Johar Town, Lahore"],
        ["Ali Raza", "03119876543", "ali@yahoo.com", "Street 9, G-10/2, Islamabad"],
        ["Hina Malik", "03220011223", "hina@outlook.com", "Flat 22, Clifton Block 5, Karachi"],
        ["Zain Ahmed", "03335556677", "zain@gmail.com", "Street 4, Satellite Town, Rawalpindi"],
        ["Sara Iqbal", "03441112223", "sara@yahoo.com", "Street 7, Model Town, Lahore"],
        ["Usman Butt", "03557778899", "usman@yahoo.com", "Near Mall Road, Peshawar"]
]

    for contact_data in contacts:
        name = contact_data[0]
        phone = contact_data[1]
        email = contact_data[2]
        address = contact_data[3]

        try:
            contact = Contact(name, phone, email, address)
            storage.add(contact)
            print("Added: " + name)
        except Exception as e:
            print("Error adding " + name + ": " + str(e))

    print("--- Sample Data Loaded Successfully ---\n")


In [98]:
def choose_data_source(storage, file_handler, manager):
    while True:
        print("How do you want to load data?")
        print("1. Use sample data")
        print("2. Load from file")
        choice = input("Choice: ")

        if choice == "1":
            create_sample_data(storage)
            print("Sample data loaded successfully.\n")
            break

        elif choice == "2":
            filename = input("Enter filename to load: ")
            try:
                result = file_handler.load_from_file(filename)
                if result:
                    print("Data loaded successfully from file.\n")
                    break
                else:
                    print("Failed to load data (file missing or corrupted).")
                    revert = input("Do you want to use sample data instead? (y/n): ").lower()
                    if revert == "y":
                        create_sample_data(storage)
                        print("Sample data loaded successfully.\n")
                        break
                    else:
                        print("Please try again.")
            except Exception as e:
                print("Error loading file:", str(e))
                revert = input("Do you want to use sample data instead? (y/n): ").lower()
                if revert == "y":
                    create_sample_data(storage)
                    print("Sample data loaded successfully.\n")
                    break
                else:
                    print("Please try again.")

        else:
            print("Invalid choice, please enter 1 or 2.\n")


In [ ]:
def main():
    storage = Storage()
    file_handler = FileHandler(storage)
    manager = ContactManager(storage, file_handler)

    choose_data_source(storage, file_handler, manager)

    menu = Menu(manager)
    menu.run()

main()
